In [ ]:
# default_exp losses
# all_slow

# Loss functions

> Various loss functions in PyTorch

In [2]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
# export
import torch
import torch.nn as nn
import torch.nn.functional as F
from fastai.torch_core import TensorBase
from fastai.losses import *
from functools import partial

In [3]:
from fastai.vision.all import *
import numpy as np
from torch.nn.modules.loss import _Loss
import segmentation_models_pytorch as smp
from steel_segmentation.utils import get_train_df
from steel_segmentation.transforms import SteelDataBlock, SteelDataLoaders

In [4]:
path = Path("../data")
train_pivot = get_train_df(path=path, pivot=True)
block = SteelDataBlock(path)
dls = SteelDataLoaders(block, train_pivot, bs=8)
xb, yb = dls.one_batch()
print(xb.shape, xb.device)
print(yb.shape, yb.device)

C:\Users\beanTech\miniconda3\envs\steel_segmentation\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


torch.Size([8, 3, 224, 1568]) cuda:0
torch.Size([8, 4, 224, 1568]) cpu


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [20]:
model = smp.Unet("resnet18", classes=4).to(device)

logits = model(xb)
probs = torch.sigmoid(logits)
preds = ( probs > 0.5).float()

In [9]:
# export
#reference: https://github.com/asanakoy/kaggle_carvana_segmentation/blob/master/asanakoy/losses.py
class SoftDiceLoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, logits, labels):
        probs = torch.sigmoid(logits)
        num = labels.size(0)
        m1 = probs.view(num, -1)
        m2 = labels.view(num, -1)
        intersection = (m1 * m2)
        score = 2. * (intersection.sum(1) + 1) / (m1.sum(1) + m2.sum(1) + 1)
        score = 1 - score.sum() / num
        return score

In [21]:
criterion = SoftDiceLoss()
criterion(logits.detach().cpu(), yb)

TensorImage(0.9844)

In [12]:
# export
#reference: https://github.com/zdaiot/Kaggle-Steel-Defect-Detection
class WeightedSoftDiceLoss(nn.Module):

    def __init__(self, size_average=True, weight=[0.2, 0.8]):
        super().__init__()
        self.size_average = size_average
        self.weight = torch.FloatTensor(weight)

    def forward(self, logit_pixel, truth_pixel):
        batch_size = len(logit_pixel)
        logit = logit_pixel.view(batch_size, -1)
        truth = truth_pixel.view(batch_size, -1)
        assert(logit.shape == truth.shape)

        loss = self.soft_dice_criterion(logit, truth)

        if self.size_average:
            loss = loss.mean()
        return loss

    def soft_dice_criterion(self, logit, truth):
        batch_size = len(logit)
        probability = torch.sigmoid(logit)

        p = probability.view(batch_size, -1)
        t = truth.view(batch_size, -1)

        w = truth.detach()
        self.weight = self.weight.type_as(logit)
        w = w * (self.weight[1] - self.weight[0]) + self.weight[0]

        p = w * (p*2 - 1)  #convert to [0,1] --> [-1, 1]
        t = w * (t*2 - 1)

        intersection = (p * t).sum(-1)
        union =  (p * p).sum(-1) + (t * t).sum(-1)
        dice  = 1 - 2 * intersection/union

        loss = dice
        return loss

In [22]:
criterion = WeightedSoftDiceLoss()
criterion(logits.detach().cpu(), yb)

TensorMask(0.9413)

In [15]:
# export
#reference: https://github.com/zdaiot/Kaggle-Steel-Defect-Detection
class SoftBCEDiceLoss(nn.Module):

    def __init__(self, size_average=True, weight=[0.2, 0.8]):
        super().__init__()
        self.size_average = 'mean' if size_average else 'none'
        self.weight = weight
        self.bce_loss = nn.BCEWithLogitsLoss(
            reduction=self.size_average, 
            pos_weight=torch.tensor(self.weight[1])
        )
        self.softdiceloss = WeightedSoftDiceLoss(
            size_average=self.size_average, 
            weight=weight
        )

    def forward(self, input, target):
        input, target = TensorBase(input).float(), TensorBase(target).float()
        soft_bce_loss = self.bce_loss(input, target)
        soft_dice_loss = self.softdiceloss(input, target)
        loss = 0.7 * soft_bce_loss + 0.3 * soft_dice_loss

        return loss

In [23]:
criterion = SoftBCEDiceLoss()
criterion(logits.detach().cpu(), yb)

TensorBase(0.8054)

In [18]:
# export
#reference: https://github.com/zdaiot/Kaggle-Steel-Defect-Detection
class MultiClassesSoftBCEDiceLoss(nn.Module):

    def __init__(self, classes_num=4, size_average=True, weight=[0.2, 0.8]):
        super().__init__()
        self.classes_num = classes_num
        self.size_average = size_average
        self.weight = weight
        self.soft_bce_dice_loss = SoftBCEDiceLoss(size_average=self.size_average, weight=self.weight)

    def forward(self, input, target):
        """
        Args:
            input: tensor, [batch_size, classes_num, height, width]
            target: tensor, [batch_size, classes_num, height, width]
        """
        loss = 0
        for class_index in range(self.classes_num):
            input_single_class = input[:, class_index, :, :]
            target_singlt_class = target[:, class_index, :, :]
            single_class_loss = self.soft_bce_dice_loss(input_single_class, target_singlt_class)
            loss += single_class_loss

        loss /= self.classes_num

        return loss

In [24]:
criterion = MultiClassesSoftBCEDiceLoss()
criterion(logits.detach().cpu(), yb)

TensorBase(0.8063)